In [5]:
from pathlib import Path
import polars as pl

INCLUDED_LABS: set[str] = {'BS1', 'BS2', 'BS3', 'BS4',
                           'EM1', 'EM2', 'EM3', 'EM4',
                           'PS1', 'PS2', 'PS3', 'RR1'}
FQ_STAT_SCHEMA = {
    'file': pl.String, 'format': pl.String,
    'type': pl.String, 'num_seqs': pl.Int64,
    'sum_len': pl.Int64, 'min_len': pl.Int64,
    'avg_len': pl.Float64, 'max_len': pl.Int64,
    'Q1': pl.Int64, 'Q2': pl.Int64, 'Q3': pl.Int64,
    'sum_gap': pl.Int64, 'N50': pl.Int64, 'N50_num': pl.Int64,
    'Q20(%)': pl.Float64, 'Q30(%)': pl.Float64,
    'AvgQual': pl.Float64, 'GC(%)': pl.Float64, 'sum_n': pl.Int64}

fq_stat_file: Path = Path('/mnt/eqa/zhangyuanfeng/methylation/evaluated/fq_stat.csv')
quality_dir: Path = Path('/mnt/eqa/zhangyuanfeng/methylation/data_for_plot/1_data_quality_snr')
cr_bias_dir: Path = Path('/mnt/eqa/zhangyuanfeng/methylation/data_for_plot/2_conversion_rate_bias')
accuracy_dir: Path = Path('/mnt/eqa/zhangyuanfeng/methylation/data_for_plot/3_quantative_rmse_spearmanr')

fq_stat: pl.DataFrame = (pl.read_csv(fq_stat_file,
                                     schema=FQ_STAT_SCHEMA)
                           .with_columns(pl.col('file')
                                           .str
                                           .head(8)
                                           .alias('sample'))
                           .with_columns(pl.col('sample')
                                           .str
                                           .head(3)
                                           .alias('lab'),
                                         pl.col('sample')
                                           .str
                                           .slice(4, 2)
                                           .alias('label'),
                                         pl.col('sample')
                                           .str
                                           .tail(1)
                                           .cast(pl.Int64)
                                           .alias('rep'))
                           .select('lab', 'label', 'rep', 'avg_len',
                                   'Q20(%)', 'Q30(%)', 'AvgQual',
                                   'GC(%)', 'sum_n'))
map_dup_rates: pl.DataFrame = (pl.read_csv(quality_dir / 'mapping_duplicate_rates.csv')
                                 .with_columns(pl.col('sample').str.head(2).alias('label'),
                                               pl.col('sample').str.tail(1).cast(pl.Int64).alias('rep'))
                                 .select('lab', 'label', 'rep',
                                         'aligned%', 'paired%', 'duplicate%'))
cytosine_efficiency: pl.DataFrame = (pl.read_csv(quality_dir / 'cytosine_efficiency.csv')
                                       .filter(pl.col('lab').is_in(INCLUDED_LABS))
                                       .filter(~pl.col('label').str.contains('HF'))
                                       .select('lab', 'label', 'rep', 'cytosine_efficiency'))
depth_efficiency: pl.DataFrame = (pl.read_csv(quality_dir / 'depth_efficiency.csv')
                                    .filter(pl.col('lab').is_in(INCLUDED_LABS))
                                    .filter(~pl.col('label').str.contains('HF'))
                                    .select('lab', 'label', 'rep', 'depth_efficiency'))
depth_percent: pl.DataFrame = (pl.read_csv(quality_dir / 'depth_5_10_percent.csv')
                                 .filter(pl.col('lab').is_in(INCLUDED_LABS))
                                 .filter(~pl.col('label').str.contains('HF'))
                                 .select('lab', 'label', 'rep', '≥5x%', '≥10x%'))
snr: pl.DataFrame = (pl.read_csv(quality_dir / 'unified_snr.csv')
                       .select('lab', 'SNR'))
cr: pl.DataFrame = (pl.read_csv(cr_bias_dir / 'cr_stat_sample.csv')
                      .filter(pl.col('lab').is_in(INCLUDED_LABS))
                      .filter(~pl.col('sample').str.contains('HF'))
                      .with_columns(pl.when(pl.col('lab').str.contains('PS'))
                                      .then(pl.col('pUC19'))
                                      .otherwise(pl.col('chrM_avg_beta'))
                                      .alias('conversion_sufficiency'),
                                    pl.when(pl.col('lab').str.contains('PS'))
                                      .then(pl.col('chrM_avg_beta'))
                                      .otherwise(pl.col('pUC19'))
                                      .alias('excessive_conversion'),
                                    pl.col('sample').str.head(2).alias('label'),
                                    pl.col('sample').str.tail(1).cast(pl.Int64).alias('rep'))
                      .select('lab', 'label', 'rep', 'conversion_sufficiency', 'excessive_conversion'))
hf_bias: pl.DataFrame = (pl.read_csv(cr_bias_dir / 'hf_bias_stats.csv')
                           .filter(pl.col('lab').is_in(INCLUDED_LABS))
                           .pivot(on='direction', index='lab', values='median_bias'))
rmse: pl.DataFrame = (pl.read_csv(accuracy_dir / 'unified_rmse.csv')
                        .filter(pl.col('fgroup') == '≥10x')
                        .filter(pl.col('lab').is_in(INCLUDED_LABS))
                        .with_columns(pl.col('sample').str.head(2).alias('label'),
                                      pl.col('sample').str.tail(1).cast(pl.Int64).alias('rep'))
                        .select('lab', 'label', 'rep', 'rmse', 'count'))
spearmanr: pl.DataFrame = (pl.read_csv(accuracy_dir / 'unified_spearmanr.csv')
                             .filter(pl.col('lab').is_in(INCLUDED_LABS))
                             .select('lab', 'label', 'spearmanr'))
consistency_np1: pl.DataFrame = (pl.read_csv(accuracy_dir / 'consistency' / 'consistency.csv')
                                   .filter(pl.col('lab1').is_in(INCLUDED_LABS),
                                           pl.col('lab2') == 'NP1')
                                   .rename({'lab1': 'lab', 'jaccard': 'Jaccard_NP1'})
                                   .select('lab', 'label', 'rep', 'Jaccard_NP1'))
consistency_ma3: pl.DataFrame = (pl.read_csv(accuracy_dir / 'consistency' / 'consistency.csv')
                                   .filter(pl.col('lab1').is_in(INCLUDED_LABS),
                                           pl.col('lab2') == 'MA3')
                                   .rename({'lab1': 'lab', 'spearmanr': 'SpearmanR_MA3'})
                                   .select('lab', 'label', 'rep', 'SpearmanR_MA3'))

In [9]:
joined: pl.DataFrame

joined = (rmse.join(other=spearmanr, on=['lab', 'label'], how='left', validate='m:1')
              .join(other=snr, on=['lab'], how='left', validate='m:1')
              .join(other=hf_bias, on=['lab'], how='left', validate='m:1')
              .join(other=cr, on=['lab', 'label', 'rep'], how='left', validate='1:1')
              .join(other=depth_percent, on=['lab', 'label', 'rep'], how='left', validate='1:1')
              .join(other=depth_efficiency, on=['lab', 'label', 'rep'], how='left', validate='1:1')
              .join(other=cytosine_efficiency, on=['lab', 'label', 'rep'], how='left', validate='1:1')
              .join(other=fq_stat, on=['lab', 'label', 'rep'], how='left', validate='1:1')
              .join(other=map_dup_rates, on=['lab', 'label', 'rep'], how='left', validate='1:1')
              .join(other=consistency_np1, on=['lab', 'label', 'rep'], how='left', validate='1:1')
              .join(other=consistency_ma3, on=['lab', 'label', 'rep'], how='left', validate='1:1')
              .rename({'rmse': 'RMSE', 'count': 'Overlap_Reference', 'spearmanr': 'SpearmanR', 'over-estimate': 'Median_Over_Estimation',
                       'under-estimate': 'Median_Under_Estimation', 'conversion_sufficiency': 'Conversion_Sufficiency',
                       'excessive_conversion': 'Excessive_Conversion', '≥5x%': 'depth≥5x(%)',
                       '≥10x%': 'depth≥10x(%)', 'depth_efficiency': 'Depth_Efficiency',
                       'cytosine_efficiency': 'Cytosine_Efficiency', 'avg_len': 'Mean_Reads_Length',
                       'GC(%)': 'GC%', 'sum_n': 'Ambiguous_Characters'})
              .with_columns(pl.concat_str(pl.col('lab'),
                                          pl.col('label'),
                                          pl.col('rep'),
                                          separator='_')
                              .alias('sample'))
              .with_columns(pl.when(pl.col('lab').str.contains('PS'))
                              .then(pl.lit(1))
                              .when(pl.col('lab').str.contains('EM'))
                              .then(pl.lit(2))
                              .otherwise(pl.lit(3))
                              .alias('Base_Conversion_Method'),
                            pl.when(pl.col('lab').str.contains('RR'))
                              .then(pl.lit(1))
                              .otherwise(pl.lit(0))
                              .alias('Targeted_Enrichment'))
              .sort('sample')
              .select('sample', 'RMSE', 'Overlap_Reference',
                      'Base_Conversion_Method', 'Targeted_Enrichment',
                      'SpearmanR', 'Jaccard_NP1', 'SpearmanR_MA3',
                      'aligned%', 'paired%', 'duplicate%',
                      'Cytosine_Efficiency', 'Depth_Efficiency', 'depth≥5x(%)', 'depth≥10x(%)',
                      'Conversion_Sufficiency', 'Excessive_Conversion',
                      'Median_Over_Estimation', 'Median_Under_Estimation', 'SNR',
                      'Mean_Reads_Length', 'Q20(%)', 'Q30(%)', 'AvgQual',
                      'GC%', 'Ambiguous_Characters'))

In [10]:
joined

sample,RMSE,Overlap_Reference,Base_Conversion_Method,Targeted_Enrichment,SpearmanR,Jaccard_NP1,SpearmanR_MA3,aligned%,paired%,duplicate%,Cytosine_Efficiency,Depth_Efficiency,depth≥5x(%),depth≥10x(%),Conversion_Sufficiency,Excessive_Conversion,Median_Over_Estimation,Median_Under_Estimation,SNR,Mean_Reads_Length,Q20(%),Q30(%),AvgQual,GC%,Ambiguous_Characters
str,f64,i64,i32,i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64
"""BS1_BC_1""",13.809189,22841865,3,0,0.712524,0.517877,0.777013,99.983416,99.988251,4.7138,10.111232,11.305179,78.84302,43.30167,0.510245,4.470985,8.333333,10.0,26.791314,150.0,98.02,94.94,25.43,21.56,1415983
"""BS1_BC_2""",13.956052,20475800,3,0,0.712524,0.46304,0.758902,99.978071,99.989093,6.8414,9.648508,10.29397,73.022662,38.79092,0.461833,4.360732,8.333333,10.0,26.791314,150.0,97.59,93.6,24.42,20.66,7155045
"""BS1_BL_1""",13.228193,25925701,3,0,0.909086,0.533882,0.911927,99.965869,99.988833,5.8039,10.2447,11.529582,83.990811,46.36178,0.461456,4.495239,8.333333,10.0,26.791314,150.0,97.56,93.15,24.54,21.24,2127339
"""BS1_BL_2""",13.159559,24263587,3,0,0.909086,0.49851,0.91263,99.965617,99.978782,4.6338,10.706092,11.426795,81.196031,42.553853,0.443768,4.741055,8.333333,10.0,26.791314,150.0,97.35,93.86,24.03,21.1,789569
"""BS1_D5_1""",16.271192,17736966,3,0,0.85921,0.447536,0.894235,99.982941,99.994386,5.4569,9.750731,10.331637,75.784523,42.475024,0.503419,4.360995,8.333333,10.0,26.791314,150.0,98.11,94.94,25.49,20.92,7790347
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""RR1_T2_2""",15.48539,1499888,3,1,0.933114,0.087052,0.891798,99.900241,99.89542,65.62,18.615726,40.30991,71.091969,57.888625,0.026155,4.842487,10.869565,12.032086,27.168239,79.3,98.18,95.09,25.49,31.24,35866
"""RR1_T3_1""",14.730816,1062958,3,1,0.941582,0.072133,0.880259,99.938041,99.932262,68.05,14.528805,42.777031,76.2025,66.960019,0.091431,4.877513,10.869565,12.032086,27.168239,74.9,98.03,95.05,25.0,33.48,32692
"""RR1_T3_2""",15.259313,1603572,3,1,0.941582,0.091517,0.902373,99.876122,99.869878,64.09,18.291305,42.506925,70.361614,59.207288,0.078029,5.315808,10.869565,12.032086,27.168239,81.2,97.76,94.12,24.59,31.4,38639


In [11]:
joined.write_csv(accuracy_dir / 'joined_metrics.csv')